In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


from internvl import InternVL
from clip_retrieval import RAGWithCLIP

# clip = RAGWithCLIP("openai/clip-vit-base-patch32")
clip = RAGWithCLIP("openai/clip-vit-large-patch14-336")

# vlm =  InternVL('OpenGVLab/InternVL2-1B')
vlm =  InternVL('OpenGVLab/InternVL2-8B')

In [ ]:
import os
import json
import glob
import argparse
from pathlib import Path
import sys
import numpy as np

dataset = '../data/open-eqa-v0.json'
# dataset = '../data/caree-eqa-v0.json'

def get_frames(episode_history):
    if "open-eqa-v0" in dataset:
        frames = sorted(glob.glob(f"/share/open-eqa/frames/{episode_history}/*-rgb.png"))

    elif "caree-eqa-v0" in dataset:
        frames = sorted(glob.glob(f"/share/open-eqa/frames/{episode_history}/*-rgb.png"))

    return frames


def get_video_path(episode_history):
    # convert Path to string
    if "open-eqa-v0" in dataset:
        video_path = f"/share/open-eqa/videos/{episode_history}-0.mp4"
    elif "caree-eqa-v0" in dataset:
        #  "episode_history": "caree-v0/caree-1"
        video_path = f"/share/care-e/testing_videos/{episode_history.split('/')[-1]}.mp4"
    else:
        assert False, "Unknown dataset"
    return video_path

eqa_data = json.load(open(dataset))[-500:]

last_episode_history  = None
for i, item in enumerate(eqa_data):
    q = item["question"]
    g = item["answer"]

    # q = 'I want to learn to code'
    # g = 'N/A'

    episode_history = item["episode_history"]

    if last_episode_history != episode_history:
        frames = get_frames(episode_history)
        print(f"Encoding {len(frames)} frames for {episode_history}")
        clip.encode_frames(frames, batch_size=8)
        last_episode_history = episode_history

    for k, v in item.items():
        print(f"{k:20}\t{v}")

    image_paths = clip.search([q], n=5, do_visualization=True)[0]

    prompt = f"""
You are an intelligent question answering agent. I will ask you questions about an indoor space and you must provide an answer.
You will be shown a set of images that have been collected from a single location.
Given a user query, you must output direct and concise one-sentence answer to the question asked by the user.
Images: {"<image>" * len(image_paths)}
User Query: {q}"""

    answer, _ = vlm(prompt=prompt, image_paths=image_paths)
    print(f'Question:    \t{q}')
    print(f'Ground truth:\t{g}')
    print(f'Prediction:  \t{answer}')
    print('= '*128)


    # break